In [7]:
import copy
import multiprocessing as mp
import multiprocessing.dummy as dummy_mp
import os
import re
from datetime import datetime
from string import Template

import pymorphy2
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

FB_LOGIN = '89026051080'
FB_PASSWORD = 'z6A-8Wd-XiV-VNY'
login, password = FB_LOGIN, FB_PASSWORD

__base_url = 'https://www.facebook.com'
__base_group_url = 'https://www.facebook.com/groups'
__login_url = 'https://www.facebook.com/login'

options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")

options.add_argument("--disable-software-rasterizer")  # maybe not needed
options.add_argument("--disable-dev-shm-usage")  # maybe not needed

browser = webdriver.Chrome(options = options)

In [10]:
def enter_facebook(browser, login, password):
    """Login to fb

    :param login: (str) fb login
    :param password: (str) fb password

    :return: -1 if error
    """
    browser.get(__login_url)

    try:
        login_form = browser.find_element_by_xpath("//input[@id='email' or @name='email']")
        login_form.clear()
        login_form.send_keys(login)
    except Exception as e:
        print(e)
        print('error: email')
        return -1

    try:
        pass_form = browser.find_element_by_xpath("//input[@id='pass' or @name='pass']")
        pass_form.clear()
        pass_form.send_keys(password)
    except Exception as e:
        print(e)
        print('error: password')
        return -1

    try:
        browser.find_element_by_xpath("//button[@id='loginbutton']").click()
    except Exception as e:
        print(e)
        print('error: login button')
        return -1

    if 'login_attempt' in browser.current_url:
        return -1

In [11]:
enter_facebook(browser, FB_LOGIN, FB_PASSWORD)

In [12]:
# posts = fb.get_posts('100032357613324')
page_id = '100032357613324'
browser.get(__base_url + '/' + page_id)

page_soup = BeautifulSoup(browser.page_source, 'lxml')

In [15]:
nonavbl = page_soup.find('title', text=re.compile('.*Страница не найдена.*'))
if nonavbl:
    print('Страница не найдена')
nonavbl = page_soup.find('h2', text=re.compile('.*Эта страница недоступна.*'))
if nonavbl:
    print('Эта страница недоступна')
nonavbl = page_soup.find('h2', text=re.compile('.*К сожалению, этот контент сейчас недоступен.*'))
if nonavbl:
    print('К сожалению, этот контент сейчас недоступен')

In [16]:
page_soup.find('title', text=re.compile('.*Страница не найдена.*'))

In [22]:
from time import sleep 

In [23]:
posts_list = []
error_count = 0
for i in range(3):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(1)

In [29]:
try:
    timeline = WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, '//div[@data-pagelet="ProfileTimeline"]')))
except TimeoutException:
    print(posts_list)
    

In [ ]:
browser.find_elements_by_xpath('//*[@id="mount_0_0_sv"]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div[2]/div/div/div/div/div/div[4]/div[2]/div/div[2]/div[3]/div[2]')

In [27]:
# browser.find_element_by_xpath("//*[@id="mount_0_0_sv"]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div[2]/div/div/div/div/div/div[4]/div[2]/div/div[2]/div[3]")
browser.find_element_by_xpath('//div[@data-pagelet="ProfileTimeline"]')

[<selenium.webdriver.remote.webelement.WebElement (session="3dade6c9669cc5cadd569c23b8393bb5", element="9b9e055d-02e1-490c-ba24-cdc5c5dbfb36")>]

In [ ]:
while len(posts_list) < limit:
    flag = browser.execute_script(js_post_prl_sqript.substitute(XPATH=post_prl_xpath))
    while flag != -1:
        flag = browser.execute_script(js_post_prl_sqript.substitute(XPATH=post_prl_xpath))

    if need_comments:
        # flag = browser.try_click_by_xpath(comment_area_prl_xpath)
        # while flag != -1:
        #    flag = browser.try_click_by_xpath(comment_area_prl_xpath)

        flag = browser.try_click_by_xpath(comment_prl_xpath)
        while flag != -1:
            flag = browser.try_click_by_xpath(comment_prl_xpath)

    try:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        posts_list = browser.find_elements_by_xpath(post_xpath)
        if posts_list is None:
            return []

        if from_date is not None and posts_list:
            last_post = Post('some', BeautifulSoup(posts_list[-1].get_attribute('innerHTML')))
            if last_post.date is not None:
                if last_post.date.timestamp() < from_date:
                    return posts_list

    except Exception as e:
        print(e)
        error_count += 1
        if error_count > 100:
            if posts_list is None:
                return []

            return posts_list

if posts_list is None:
    return []